In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import friedmanchisquare
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from itertools import combinations
import openpyxl

In [2]:
# Define the file name and path
file_name = "Cleaned-Data.csv"
file_path = os.path.join(os.getcwd(), file_name)


# Read the CSV file with semicolon delimiter
df = pd.read_csv(file_path, delimiter=';')

In [3]:
# List the columns you want to remove
columns_to_remove = ['Date']  # Replace with your actual column names

# Drop the specified columns
df = df.drop(columns=columns_to_remove)

In [4]:
# Filter the DataFrame to only include rows where the "filter" column is one of "A", "B", "C", "D"
df_filtered = df[df['Filter'].isin(["A", "B", "C", "D"])]

# Sort the DataFrame by "filter" first (grouping) and then by "Number" within each "filter" group
df = df_filtered.sort_values(by=["Filter", "Number"], ascending=[True, True])

In [5]:
# Removing all der ikke har været der alle 4 gange
# List of participant numbers to keep
participants_to_keep = [
    1, 2, 4, 6, 7, 8, 9, 10, 11, 15, 13, 14, 16, 17, 18, 19,
    20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35,
    36, 37, 38, 39, 40, 42, 43, 45, 46, 47, 49, 50, 51, 52,
    53, 54, 55, 56, 57, 58, 59, 60, 62, 64, 65, 66, 67, 68, 
    69
]

# Filter the DataFrame
df = df[df['Number'].isin(participants_to_keep)]

In [6]:
# List the columns you want to remove
columns_to_remove = ['Number']  # Replace with your actual column names

# Drop the specified columns
df = df.drop(columns=columns_to_remove)

In [7]:
#defining dataframes
df_A = df[df['Filter'] == 'A']
df_B = df[df['Filter'] == 'B']
df_C = df[df['Filter'] == 'C']
df_D = df[df['Filter'] == 'D']



In [8]:
# DataFrames for each dataset
dataframes = {
    "Hepa 7": df_A,
    "Hepa 5": df_C,
    "APSU": df_B,
    "APSN": df_D
}

In [9]:
#Freidman willcoxon
#over time / by questionair
output_path = f'Friedman-Wilcoxon-by-Questionair-nasal-humidity.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": df_A,
    "Hepa 5": df_C,
    "APSU": df_B,
    "APSN": df_D
}

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for dataset_name, df in dataframes.items():
        # Separate columns for H (humidity) and F (flow)
        humidity_columns = [col for col in df.columns if col.endswith("H_avg")]
        flow_columns = [col for col in df.columns if col.endswith("F_avg")]

        # Process each group separately
        for group_name, group_columns in [("Humidity", humidity_columns), ("Flow", flow_columns)]:
            # Initialize results DataFrames
            friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant"])
            wilcoxon_results_list = []

            for col in group_columns:
                # Extract Medical Test
                medical_test_groups = []
                for test in ["M1", "M2", "M3"]:
                    test_col = f"{test}-{group_name[0]}_avg"  # E.g., M1-H_avg for Humidity
                    if test_col in df.columns:
                        medical_test_groups.append(df[test_col].dropna().values)

                # Ensure at least two groups have data
                if len(medical_test_groups) >= 2:
                    # Perform the Friedman test
                    friedman_stat, friedman_p = friedmanchisquare(*medical_test_groups)
                    is_significant = 1 if friedman_p < alpha else 0

                    # Append Friedman test results
                    temp_df = pd.DataFrame({
                        "Column": [col],
                        "Statistic": [friedman_stat],
                        "p-value": [friedman_p],
                        "Alpha": [alpha],
                        "Significant": [is_significant]
                    })
                    friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)

                    # Perform pairwise Wilcoxon tests
                    if is_significant:
                        pairs = combinations(range(len(medical_test_groups)), 2)
                        for i, j in pairs:
                            group1, group2 = medical_test_groups[i], medical_test_groups[j]
                            if len(group1) > 0 and len(group2) > 0:
                                wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                                wilcox_significant = 1 if wilcox_p < alpha else 0
                                wilcoxon_results_list.append({
                                    "Column": col,
                                    "Pair": f"M{i+1}-M{j+1}",
                                    "Statistic": wilcox_stat,
                                    "p-value": wilcox_p,
                                    "Alpha": alpha,
                                    "Significant": wilcox_significant
                                })

            # Save results for the current group
            friedman_results_df.to_excel(writer, sheet_name=f"{dataset_name} {group_name} Friedman Results", index=False)
            if wilcoxon_results_list:
                wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
                wilcoxon_df.to_excel(writer, sheet_name=f"{dataset_name} {group_name} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.")


C:\Users\ejnar\AppData\Local\Temp\ipykernel_4864\3689001800.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\OneDrive\DTU\speciale_projekt\venv\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\ejnar\AppData\Local\Temp\ipykernel_4864\3689001800.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determini

Friedman and Wilcoxon test results saved to Friedman-Wilcoxon-by-Questionair-nasal-humidity.xlsx.


In [10]:
#Freidman willcoxon
#by condtions
output_path = f'Friedman-Wilcoxon-by-Filter-nasal-humidity.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": df_A,
    "Hepa 5": df_C,
    "APSU": df_B,
    "APSN": df_D
}

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for condition in ['H', 'F']: 
        for medical_test in range(1, 4): 
            # Prepare results DataFrames
            friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant"])
            wilcoxon_results_list = []

            # Dynamically identify relevant columns
            columns_to_test = [col for col in dataframes["Hepa 7"].columns if f"M{medical_test}-{condition}_avg" in col]

            for col in columns_to_test:
                # Collect data across datasets
                data_groups = [
                    dataframes[name][col].dropna().values
                    for name in dataframes
                ]

                # Perform Friedman test
                friedman_stat, friedman_p = friedmanchisquare(*data_groups)
                is_significant = 1 if friedman_p < alpha else 0

                # Append Friedman results
                friedman_results_df = pd.concat([
                    friedman_results_df,
                    pd.DataFrame({
                        "Column": [col],
                        "Statistic": [friedman_stat],
                        "p-value": [friedman_p],
                        "Alpha": [alpha],
                        "Significant": [is_significant]
                    })
                ], ignore_index=True)

                # Perform pairwise Wilcoxon
                if is_significant:
                    pairs = combinations(dataframes.keys(), 2)
                    for name1, name2 in pairs:
                        group1 = dataframes[name1][col].dropna().values
                        group2 = dataframes[name2][col].dropna().values

                        if len(group1) > 0 and len(group2) > 0:
                            wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                            wilcox_significant = 1 if wilcox_p < alpha else 0

                            # Append Wilcoxon results
                            wilcoxon_results_list.append({
                                "Column": col,
                                "Pair": f"{name1} vs {name2}",
                                "Statistic": wilcox_stat,
                                "p-value": wilcox_p,
                                "Alpha": alpha,
                                "Significant": wilcox_significant
                            })

            # Save Friedman results
            friedman_results_df.to_excel(writer, sheet_name=f"{condition}-M{medical_test}-Friedman", index=False)

            # Save Wilcoxon results
            if wilcoxon_results_list:
                wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
                wilcoxon_df.to_excel(writer, sheet_name=f"{condition}-M{medical_test}-Wilcoxon", index=False)

print(f"Results saved to {output_path}.")


C:\Users\ejnar\AppData\Local\Temp\ipykernel_4864\1119838483.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_4864\1119838483.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_4864\1119838483.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer

Results saved to Friedman-Wilcoxon-by-Filter-nasal-humidity.xlsx.


In [11]:
# Define the filename
#save dataframe
filename = "Humid_nasal_output.xlsx"

# Get the current working directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, filename)




# Initialize an Excel writer
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Loop through each dataset and save it to a separate sheet
    for dataset_name, df in dataframes.items():
        # Save each DataFrame to its corresponding sheet
        df.to_excel(writer, sheet_name=dataset_name, index=False)

print(f"DataFrames saved to {file_path}")

DataFrames saved to C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Medical-data - humidity and nasal\Humid_nasal_output.xlsx
